# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atafu village,-8.5421,-172.5159,28.03,72,100,8.41,TK,1736389846
1,1,blackmans bay,-43.0167,147.3167,23.62,48,100,4.37,AU,1736389847
2,2,port-aux-francais,-49.3500,70.2167,6.64,98,100,11.81,TF,1736389849
3,3,punta arenas,-53.1500,-70.9167,8.06,66,20,10.29,CL,1736389702
4,4,bethel,41.3712,-73.4140,-5.95,59,0,4.63,US,1736389851


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat",
    geo=True,
    size='Humidity', 
    tiles="OSM", 
    color="City",
    scale=0.5,
    frame_width=700,
    frame_height=400
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = pd.read_csv("output_data/cities.csv")

# Drop any rows with null values
city_data_df_cleaned = city_data_df.dropna()

# Display sample data
city_data_df_cleaned.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atafu village,-8.5421,-172.5159,28.03,72,100,8.41,TK,1736389846
1,1,blackmans bay,-43.0167,147.3167,23.62,48,100,4.37,AU,1736389847
2,2,port-aux-francais,-49.3500,70.2167,6.64,98,100,11.81,TF,1736389849
3,3,punta arenas,-53.1500,-70.9167,8.06,66,20,10.29,CL,1736389702
4,4,bethel,41.3712,-73.4140,-5.95,59,0,4.63,US,1736389851


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,atafu village,TK,-8.5421,-172.5159,72,
1,blackmans bay,AU,-43.0167,147.3167,48,
2,port-aux-francais,TF,-49.3500,70.2167,98,
3,punta arenas,CL,-53.1500,-70.9167,66,
4,bethel,US,41.3712,-73.4140,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000  # in meters

# Define parameters for the Geoapify API request
params = {
    'categories': 'accommodation.hotel',  # Searching for hotels
    'apiKey': geoapify_key,  # Replace with your actual Geoapify API key
    'limit': 1,  # We want only the first result (the nearest hotel)
}

# Print a message to indicate the start of the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"  # Search within the specified radius
    params["bias"] = f"proximity:{longitude},{latitude}"  # Bias the search towards the city center

    # Set base URL for Geoapify API request
    base_url = "https://api.geoapify.com/v2/places"

    # Make the API request using the params dictionary
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Check for any errors with the request

        # Convert the API response to JSON format
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_name = name_address["features"][0]["properties"]["name"]
            hotel_df.loc[index, "Hotel Name"] = hotel_name
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found"
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    except requests.exceptions.RequestException as e:
        # Handle network errors or API request errors
        hotel_df.loc[index, "Hotel Name"] = "Error in API request"
        print(f"Error fetching hotel for {row['City']}: {e}")
    
    # Log the search result for each city
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data with hotel names populated
hotel_df.head()

Starting hotel search
atafu village - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
port-aux-francais - nearest hotel: Keravel
punta arenas - nearest hotel: Hotel Lacolet
bethel - nearest hotel: Hampton Inn Danbury
hawaiian paradise park - nearest hotel: No hotel found
tura - nearest hotel: Hotel Polo Orchid
prainha - nearest hotel: No hotel found
ta`u - nearest hotel: No hotel found
uliastay - nearest hotel: Болоржин зочид буудал
uturoa - nearest hotel: Hawaiki Nui hotel
isafjordur - nearest hotel: Hótel Horn
chonchi - nearest hotel: Hotel Huildín
vorgashor - nearest hotel: София
al mijlad - nearest hotel: No hotel found
bayevo - nearest hotel: Гостиница "Кулунда"
hermanus - nearest hotel: Aloe guest house
mariehamn - nearest hotel: Hotell Esplanad
edinburgh of the seven seas - nearest hotel: No hotel found
whangarei - nearest hotel: Grand Hotel
jamestown - nearest hotel: DoubleTree Jamestown
adamstown - nearest hotel: No hotel found
waitangi - nearest hote

,City,Country,Lat,Lng,Humidity,Hotel Name
0,atafu village,TK,-8.5421,-172.5159,72,No hotel found
1,blackmans bay,AU,-43.0167,147.3167,48,Villa Howden
2,port-aux-francais,TF,-49.3500,70.2167,98,Keravel
3,punta arenas,CL,-53.1500,-70.9167,66,Hotel Lacolet
4,bethel,US,41.3712,-73.4140,59,Hampton Inn Danbury


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
city_map = hotel_df.hvplot.points(
    "Lng",  # Longitude is used for the x-axis
    "Lat",   # Latitude is used for the y-axis
    geo=True,  # Enable geographic projection (maps the points to actual locations)
    tiles="OSM",
    size="Humidity",
    scale=0.5,
    hover_cols=['Hotel Name'],  # Hover will show these columns
    color='City',  # Color of the points corresponds to humidity level
    frame_width=700,  # Width of the map
    frame_height=400  # Height of the map
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)